# MongoDB Aggregation Pipeline In Python

Learn about the various stages and configurations you can create to configure an Aggregation Pipeline

Stages: [match](#match), [project](#project), [unset](#unset), [limit](#limit), [skip](#skip), [sort](#sort), [count](#count), [sortByCount](#sortByCount), [unwind](#unwind), [group](#group), [addFields](#addFields), [sample](#sample), [lookup](#lookup), [unionWith](#unionWith), [out](#out), [merge](#merge)

Operators: [size](#size-(operator)), [in](#in-(operator)), [arrayElemAt](#arrayElemAt-(operator)), [first](#first-(operator)), [count](#count-(accumulator-operator)), [sum](#sum-(accumulator-operator)), [first, last](#first,-last-(accumulator-operators)), [push](#push-(accumulator-operator)), [addToSet](#addToSet-(accumulator-operator)), [regexMatch](#regexMatch-(operator)), [cond](#cond-(operator)), [Date](#Date-Operators), [expr](#expr-(operator)), [ifNull](#ifNull-(operator)), [type](#type-(operator)), [switch](#switch-(operator))

In [3]:
from pymongo import MongoClient

In [4]:
mongodb_uri = "mongodb://localhost:27017/"
db_name = "aggregation_test"

In [5]:
client = MongoClient(mongodb_uri)
db = client[db_name]

### Helper Function

In [6]:
def print_cursor(cursor):
    for document in cursor:
        print(document, end="\n\n")

### Inserting Some Sample Data

In [7]:
#it double checks if there is data or not -- learn it
import aggregatehelper.insert_aggregation_sample_data as iasd

iasd.insert_data(mongodb_uri, db_name)

Entries already exist in the aggregation_test database in the users, products, or orders collection. Insert commands aborted.


### match

In [8]:
match_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}}
])

In [9]:
print_cursor(match_cursor)

{'_id': ObjectId('640b4ff43c493ec19556619b'), 'name': 'Pens', 'seller_id': ObjectId('640b4ff43c493ec195566194'), 'tags': ['Office', 'School']}



In [17]:
# returns commandCursor - like cursor Object
match_cursor = db.products.aggregate([
    {"$match": {"$or": [
        {"tags": "Beauty"},
        {"tags": "Home"}]}}
])

In [18]:
print_cursor(match_cursor)

{'_id': ObjectId('640b4ff43c493ec195566199'), 'name': 'Mug', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('640b4ff43c493ec19556619a'), 'name': 'Moisturizer', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619c'), 'name': 'Face Cleanser', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619d'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('640b4ff43c493ec195566196'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619e'), 'name': 'Eyeliner', 'seller_id': ObjectId('640b4ff43c493ec195566197'), 'tags': ['Beauty']}



### project

In [31]:
#change returned column name as you wish with $ infront of the column name you want to rename
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name_as_product_name": "$name", "tags": 1}}])

In [32]:
print_cursor(project_cursor)

{'tags': ['Home', 'Kitchen'], 'name_as_product_name': 'Mug'}

{'tags': ['Beauty'], 'name_as_product_name': 'Moisturizer'}

{'tags': ['Office', 'School'], 'name_as_product_name': 'Pens'}

{'tags': ['Beauty'], 'name_as_product_name': 'Face Cleanser'}

{'tags': ['Beauty'], 'name_as_product_name': 'Concealer Makeup'}

{'tags': ['Beauty'], 'name_as_product_name': 'Eyeliner'}



In [43]:
match_project_cursor = db.products.aggregate([
    # copied from above match statement
    {"$match": {"$or": [{"name": "Pens"}]}},
    {"$project": {"_id": 0, "name_as_product_name": "$name", "tags": 1}}])

print_cursor(match_project_cursor)

{'tags': ['Office', 'School'], 'name_as_product_name': 'Pens'}



In [40]:
# no results
match_project_cursor = db.products.aggregate([
    {"$match": {"$or": [{"name": "Pens"}]}},
    {"$project": {"_id": 0, "name_as_product_name": "$name", "tags": 1}},
    {"$match": {"name": "Pens"}}
])

print_cursor(match_project_cursor)

In [42]:
# but if you change 2nd match to product name then it sees
match_project_cursor = db.products.aggregate([
    {"$match": {"$or": [{"name": "Pens"}]}},
    {"$project": {"_id": 0, "product_name": "$name", "tags": 1}},
    {"$match": {"product_name": "Pens"}}
])

print_cursor(match_project_cursor)

{'tags': ['Office', 'School'], 'product_name': 'Pens'}



### unset

In [47]:
# projection like , but  not passing attributes you want to keep but pass attributes you want to discard
unset_project_cursor = db.products.aggregate([
    {"$unset": ["_id", "seller_id"]}
])
print_cursor(unset_project_cursor)

{'name': 'Mug', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'tags': ['Beauty']}

{'name': 'Pens', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'tags': ['Beauty']}



In [52]:
unset_project_cursor = db.products.aggregate([
    {"$match": {"$or": [{"name": "Pens"}]}},
    {"$unset": ["_id", "seller_id"]}
])
print_cursor(unset_project_cursor)

{'name': 'Pens', 'tags': ['Office', 'School']}



### limit

In [54]:
limit_project_cursor = db.products.aggregate([
    {"$limit": 3}
])
print_cursor(limit_project_cursor)

{'_id': ObjectId('640b4ff43c493ec195566199'), 'name': 'Mug', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('640b4ff43c493ec19556619a'), 'name': 'Moisturizer', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619b'), 'name': 'Pens', 'seller_id': ObjectId('640b4ff43c493ec195566194'), 'tags': ['Office', 'School']}



### skip

In [56]:
skip_project_cursor = db.products.aggregate([
    {"$skip": 2}
])
print_cursor(skip_project_cursor)

{'_id': ObjectId('640b4ff43c493ec19556619b'), 'name': 'Pens', 'seller_id': ObjectId('640b4ff43c493ec195566194'), 'tags': ['Office', 'School']}

{'_id': ObjectId('640b4ff43c493ec19556619c'), 'name': 'Face Cleanser', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619d'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('640b4ff43c493ec195566196'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619e'), 'name': 'Eyeliner', 'seller_id': ObjectId('640b4ff43c493ec195566197'), 'tags': ['Beauty']}



In [61]:
skip_and_limit_project_cursor = db.products.aggregate([
    {"$skip": 2},
    {"$limit": 3}
])
print_cursor(skip_and_limit_project_cursor)

{'_id': ObjectId('640b4ff43c493ec19556619b'), 'name': 'Pens', 'seller_id': ObjectId('640b4ff43c493ec195566194'), 'tags': ['Office', 'School']}

{'_id': ObjectId('640b4ff43c493ec19556619c'), 'name': 'Face Cleanser', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619d'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('640b4ff43c493ec195566196'), 'tags': ['Beauty']}



In [63]:
# order matters
limit_and_skip_project_cursor = db.products.aggregate([
    {"$limit": 3},
    {"$skip": 2}
])
print_cursor(limit_and_skip_project_cursor)

{'_id': ObjectId('640b4ff43c493ec19556619b'), 'name': 'Pens', 'seller_id': ObjectId('640b4ff43c493ec195566194'), 'tags': ['Office', 'School']}



### sort

In [64]:
# Ascending order alphabetical order
sort_cursor = db.products.aggregate([
    {"$sort": {"name": 1}}
])
print_cursor(sort_cursor)

{'_id': ObjectId('640b4ff43c493ec19556619d'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('640b4ff43c493ec195566196'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619e'), 'name': 'Eyeliner', 'seller_id': ObjectId('640b4ff43c493ec195566197'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619c'), 'name': 'Face Cleanser', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec19556619a'), 'name': 'Moisturizer', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Beauty']}

{'_id': ObjectId('640b4ff43c493ec195566199'), 'name': 'Mug', 'seller_id': ObjectId('640b4ff43c493ec195566193'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('640b4ff43c493ec19556619b'), 'name': 'Pens', 'seller_id': ObjectId('640b4ff43c493ec195566194'), 'tags': ['Office', 'School']}



### count

In [68]:
count_cursor = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
    {"$count": "beauty_products_count"}
])
print_cursor(count_cursor)

{'beauty_products_count': 4}



### sortByCount

In [69]:
# sorted from highest to the lowest

sortByCount_cursor = db.products.aggregate([
    {"$sortByCount": "$tags"}
])
print_cursor(sortByCount_cursor)

{'_id': ['Beauty'], 'count': 4}

{'_id': ['Office', 'School'], 'count': 1}

{'_id': ['Home', 'Kitchen'], 'count': 1}



### size (operator)

In [75]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "num_tags": {"$size": "$tags"}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'num_tags': 2, 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Pens', 'num_tags': 2, 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Eyeliner', 'num_tags': 1, 'tags': ['Beauty']}



In [79]:
# this is same but only difference is order of printing!
# here tags :1 is showing tags firs, but if you need num_tags first displayed then use "tags": "$tags"
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "num_tags": {"$size": "$tags"}, "tags": 1}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'tags': ['Home', 'Kitchen'], 'num_tags': 2}

{'name': 'Moisturizer', 'tags': ['Beauty'], 'num_tags': 1}

{'name': 'Pens', 'tags': ['Office', 'School'], 'num_tags': 2}

{'name': 'Face Cleanser', 'tags': ['Beauty'], 'num_tags': 1}

{'name': 'Concealer Makeup', 'tags': ['Beauty'], 'num_tags': 1}

{'name': 'Eyeliner', 'tags': ['Beauty'], 'num_tags': 1}



### in (operator)

In [80]:
#which products have bauty tag within the array
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "is_beauty_product": {"$in": ["Beauty", "$tags"]}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'is_beauty_product': False, 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Pens', 'is_beauty_product': False, 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Eyeliner', 'is_beauty_product': True, 'tags': ['Beauty']}



### arrayElemAt (operator)

In [81]:
# grab value at particular index
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "first_tag": {"$arrayElemAt": ["$tags", 0]}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Home', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'Office', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



In [82]:
# if length of array is smaller than index value you specify that entries that have an array with length will not have attributes is not showed at all
# check second entryhas not first_tag attribute
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "first_tag": {"$arrayElemAt": ["$tags", 1]}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Kitchen', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'School', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'tags': ['Beauty']}



### first (operator)

In [83]:

project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "first_tag": {"$first": "$tags"}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Home', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'Office', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



In [84]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "first_tag": {"$last": "$tags"}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Kitchen', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'School', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



### unwind

In [11]:
# takes entry  that has array of values and
# create separate entry with each individual array element
# that has all other attributes in the document

# tags is list so it creates independent row for each element of the list
#like flatten

unwind_cursor = db.products.aggregate([
    {"$unwind": "$tags"},
    {"$unset": ["_id", "seller_id"]}
])
print_cursor(unwind_cursor)

{'name': 'Mug', 'tags': 'Home'}

{'name': 'Mug', 'tags': 'Kitchen'}

{'name': 'Moisturizer', 'tags': 'Beauty'}

{'name': 'Pens', 'tags': 'Office'}

{'name': 'Pens', 'tags': 'School'}

{'name': 'Face Cleanser', 'tags': 'Beauty'}

{'name': 'Concealer Makeup', 'tags': 'Beauty'}

{'name': 'Eyeliner', 'tags': 'Beauty'}



In [12]:
unwind_cursor = db.orders.aggregate([
    {"$unwind": "$items"},
    {"$unset": "_id"}
])
print_cursor(unwind_cursor)

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619a'), 'quantity': 1}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619c'), 'quantity': 1}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619d'), 'quantity': 1}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619e'), 'quantity': 1}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619b'), 'quantity': 5}}

{'items': {'product_id': ObjectId('640b4ff43c493ec195566199'), 'quantity': 1}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619a'), 'quantity': 2}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619e'), 'quantity': 1}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619a'), 'quantity': 1}}

{'items': {'product_id': ObjectId('640b4ff43c493ec19556619c'), 'quantity': 1}}



In [18]:
#use includeArrayIndex for the index of the array use "table_index" to identify index of the array

unwind_cursor = db.products.aggregate([
    {"$match": {"tags": {"$size":2}} }, # if size of tags_array >2 then show
    {"$unwind": {"path" : "$tags", "includeArrayIndex":"tag_index"}}, #create elements for each of the
    {"$unset":[ "_id", "seller_id"] }
])
print_cursor(unwind_cursor)

{'name': 'Mug', 'tags': 'Home', 'tag_index': 0}

{'name': 'Mug', 'tags': 'Kitchen', 'tag_index': 1}

{'name': 'Pens', 'tags': 'Office', 'tag_index': 0}

{'name': 'Pens', 'tags': 'School', 'tag_index': 1}



### group

### count (accumulator operator)

### sum (accumulator operator)

### first, last (accumulator operators)

### push (accumulator operator)

### addToSet (accumulator operator)

### $$ROOT (system variable)

### addFields

### sample

### lookup

### unionWith

### regexMatch (operator)

### out
*Note: You can potentially overwrite all your data in a collection with this stage, use with caution*

### merge
*Note: You can potentially overwrite data within a collection with this stage, use with caution*

### cond (operator)

### $$NOW (system variable)

### Date Operators

### expr (operator)

### ifNull (operator)

### type (operator)

### switch (operator)